In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ml_df = pd.read_csv("ML_data.csv")

In [5]:
tar_list = ['cardio_death', 'total_cancer', 'bladder', 'brain',
       'breast', 'breast_insitu', 'cervix', 'colon', 'esophagus',
       'kidney and renal', 'leukemia', 'liver', 'lung', 'melanoma',
       'non-hudgkin lymphoma', 'oral cavity', 'ovay', 'pancreas', 'prostate',
       'stomach', 'thyroid', 'uteras','Pediatric Asthma', 'Adult Asthma',
       'COPD', 'Adult Chronic Lung Disease']
feat_list=['air_quality_annual_PM2p5', 'median_household_income',
       'percent_diagnosed_diabetes', 'percent_EDU_less_col',
       'percent_EDU_less_HS', 'percent_food_stamp', 'percent_high_BP',
       'percent_high_chol', 'percent_obisity', 'percent_phys_inactivity',
       'percent_poverty', 'percent_somker_status', 'unemployment_rate',
       'distance', 'plant_capacity',
       'nameplate_capacity_MW1', 'NOx_tons1', 'SO2_tons1', 'CO2_tons1',
       'CH4_lbs1', 'N2O_lbs1', 'PM2.5_tons1', 'dist_from_county1',
       'nameplate_capacity_MW2', 'NOx_tons2', 'SO2_tons2', 'CO2_tons2',
       'CH4_lbs2', 'N2O_lbs2', 'PM2.5_tons2', 'dist_from_county2',
       'nameplate_capacity_MW3', 'NOx_tons3', 'SO2_tons3', 'CO2_tons3',
       'CH4_lbs3', 'N2O_lbs3', 'PM2.5_tons3', 'dist_from_county3',
       'nameplate_capacity_MW4', 'NOx_tons4', 'SO2_tons4', 'CO2_tons4',
       'CH4_lbs4', 'N2O_lbs4', 'PM2.5_tons4', 'dist_from_county4',
       'nameplate_capacity_MW5', 'NOx_tons5', 'SO2_tons5', 'CO2_tons5',
       'CH4_lbs5', 'N2O_lbs5', 'PM2.5_tons5', 'dist_from_county5',
       'fuel_type1_Biomass', 'fuel_type1_Coal', 'fuel_type1_Gas',
       'fuel_type1_Oil', 'fuel_type1_Other Fossil', 'fuel_type2_Biomass',
       'fuel_type2_Coal', 'fuel_type2_Gas', 'fuel_type2_Oil',
       'fuel_type2_Other Fossil', 'fuel_type3_Biomass', 'fuel_type3_Coal',
       'fuel_type3_Gas', 'fuel_type3_Oil', 'fuel_type3_Other Fossil',
       'fuel_type4_Biomass', 'fuel_type4_Coal', 'fuel_type4_Gas',
       'fuel_type4_Oil', 'fuel_type4_Other Fossil', 'fuel_type5_Biomass',
       'fuel_type5_Coal', 'fuel_type5_Gas', 'fuel_type5_Oil',
       'fuel_type5_Other Fossil']

In [6]:
high_corr={}
for i in range(len(feat_list)-1):
    high_corr[feat_list[i]] = []
    for j in range(i+1, len(feat_list)):
        if (abs(ml_df[feat_list[i]].corr(ml_df[feat_list[j]])) >= 0.90):
            high_corr[feat_list[i]].append(feat_list[j])
temp_df = ml_df
exl = []
for x in high_corr.keys():
    if (len(high_corr[x]) != 0):
        temp_df.drop(columns=x,inplace = True)
        exl.append(x)

In [7]:
X = ml_df[[x for x in feat_list if x not in exl]]
y = ml_df[tar_list]
for i in X.columns:
    change = abs(min(X[i])-max(X[i]))
    if (change >= 1000):
        X[i] = np.log10(X[i].replace(0, 1))
range_={}
for i in X.columns:
    range_[i] = abs(min(X[i])-max(X[i]))
#the following columns have just 1 so they were dropped
X.drop(columns=['fuel_type5_Other Fossil','fuel_type4_Other Fossil','fuel_type2_Other Fossil','fuel_type1_Other Fossil'],inplace=True)
corr_total_cancer={}
X_corr = X
for x in X.columns:
    corr_total_cancer[x] = X[x].corr(y["total_cancer"])
    if (abs(X[x].corr(y["total_cancer"])) < 0.2):
        X_corr.drop(columns=x,inplace=True)

C:\Users\afrap\AppData\Local\Temp\ipykernel_14664\3552899498.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = np.log10(X[i].replace(0, 1))
C:\Users\afrap\AppData\Local\Temp\ipykernel_14664\3552899498.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['fuel_type5_Other Fossil','fuel_type4_Other Fossil','fuel_type2_Other Fossil','fuel_type1_Other Fossil'],inplace=True)
C:\Users\afrap\AppData\Local\Temp\ipykernel_14664\3552899498.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 500, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
#rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores

#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 1500, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model

X_train, X_test, y_train, y_test = train_test_split(X_corr, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#rf_random.fit(X_train_scaled,y_train["total_cancer"])

#best_random = rf_random.best_estimator_
#pred = best_random.predict(X_test_scaled)
#pred_train = best_random.predict(X_train_scaled)

In [6]:
#import joblib
#joblib.dump(best_random, "./random_forest_random_corr_total_cancer.joblib")

['./random_forest_random_corr_total_cancer.joblib']

In [7]:
#best_random = rf_random.best_estimator_

In [8]:
#from sklearn.metrics import mean_squared_error, r2_score
#print([mean_squared_error(y_test["total_cancer"], pred),r2_score(y_test["total_cancer"], pred)
#                    ,r2_score(y_train["total_cancer"],pred_train)])

[7460.576378728113, 0.29113837321893143, 0.786195629099673]


In [2]:
import joblib
loaded_rf = joblib.load("./random_forest_random_corr_total_cancer.joblib")

In [3]:
loaded_rf

RandomForestRegressor(max_depth=50, max_features='sqrt', min_samples_leaf=2,
                      n_estimators=1166)

In [11]:
from sklearn.metrics import mean_squared_error, r2_score

pred = loaded_rf.predict(X_test_scaled)
pred_train = loaded_rf.predict(X_train_scaled)
print([mean_squared_error(y_test["total_cancer"], pred),r2_score(y_test["total_cancer"], pred)
                    ,r2_score(y_train["total_cancer"],pred_train)])

[7460.576378728113, 0.29113837321893143, 0.786195629099673]


In [16]:
import xgboost

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

xgb_reg = xgboost.XGBRFRegressor()
xgb_reg.fit(X_train_scaled,y_train["total_cancer"])
pred = xgb_reg.predict(X_test_scaled)
pred_train = xgb_reg.predict(X_train_scaled)
print([mean_squared_error(y_test["total_cancer"], pred),r2_score(y_test["total_cancer"], pred)
                    ,r2_score(y_train["total_cancer"],pred_train)])

[7838.588980620899, 0.25522176110763684, 0.5559771784674907]
